In [ ]:
#!pip install sqlalchemy jupysql

##  SQL vs Pandas

In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy
from pathlib import Path
%load_ext sql

In [ ]:
# Run this cell to connect to database
engine = sqlalchemy.create_engine('sqlite:///starwars.db')
connection = engine.connect()

In [ ]:
%sql engine


In [ ]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

In [ ]:
%%sql
# Display the first 5 rows of characters table
SELECT * FROM characters LIMIT 5;


In [ ]:
# Read the table into a pandas DataFrame
characters_df = pd.read_sql("SELECT * FROM characters", connection)

characters_df

In [ ]:
characters_df.dtypes

##  Lets compare some SQL to Pandas commands   

Demo 1 - Select Data ( columns)

In [ ]:
%sql SELECT name, species FROM characters

and in Pandas

In [ ]:
characters_df[['name', 'species']]

Demo 2 - Filtering

In [ ]:
%%sql
SELECT * FROM characters
WHERE  height > 200 AND species = 'Human'

Oops that didn't work - because apparently height is a string and so the boolean about hieght didnt filter

One approach is to ask SQL to treat that column as an integer

`CAST()` is a SQL function that converts one data type into another. This is useful when a column is stored as text (TEXT or VARCHAR), but you need to perform numeric operations on it.


In [ ]:
%%sql
SELECT * FROM characters
WHERE CAST(height AS INTEGER) > 200 
AND species = 'Human';

In [ ]:
characters_df

in Pandas we also need to check the type of height - if it wasnt a float in SQL it probably isnt in Pandas

In [ ]:
characters_df['height'].dtype

convert height to numeric

In [ ]:
characters_df['height'] = pd.to_numeric(characters_df['height'], errors='coerce')
characters_df['height'].dtype

In [ ]:
characters_df[(characters_df['height'] > 200) & (characters_df['species'] == 'Human')]

Demo 3 - aggregate data

Average height by species



In [ ]:
%%sql
SELECT species, AVG(CAST(height AS INTEGER)) AS avg_height
FROM characters
GROUP BY species;

And in Pandas

In [ ]:
species_avg_height = characters_df.groupby('species')['height'].mean()

species_avg_height

Demo 4 - 

4.1 Let's join two tables characters and planets

4.2 Let's look for Humans from the desert

In [ ]:
%%sql
SELECT characters.name, planets.name AS homeworld
FROM characters
JOIN planets ON characters.homeworld = planets.name;

In [ ]:
%%sql
SELECT characters.name, characters.homeworld, characters.species, planets.climate, planets.population, planets.terrain
FROM characters
JOIN planets ON characters.homeworld = planets.name;

In [ ]:
%%sql
SELECT characters.name, characters.homeworld, characters.species, planets.climate, planets.population, planets.terrain
FROM characters
JOIN planets ON characters.homeworld = planets.name
WHERE LOWER(characters.species) = 'human' 
AND LOWER(planets.terrain) = 'desert';

In [ ]:
planets_df = pd.read_sql("SELECT * FROM planets", connection)


In [ ]:
charactersXplanets= characters_df[['name', 'species','homeworld']].merge(planets_df, left_on='homeworld', right_on='name')
charactersXplanets

Get a subset of the humans from the desert from merged data


In [ ]:
charactersXplanets[(charactersXplanets['species'] == 'Human') & (charactersXplanets['terrain'] == 'desert')]
